## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default

tf.__version__

'1.13.1'

In [3]:
tf.enable_eager_execution()


### Collect Data

In [4]:
#from google.colab import drive
#drive.mount('/gdrive')

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [7]:
data.shape

(851264, 7)

In [8]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [9]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [10]:
data.drop('date', axis=1, inplace=True)

In [11]:
data.drop('symbol', axis=1, inplace=True)

In [12]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [13]:
df = data.head(1000)

In [14]:
df.shape

(1000, 5)

In [15]:
df['volume'] = df['volume']/1000000

C:\Users\hp\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
df.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [17]:
from sklearn.model_selection import train_test_split
X=df.drop("volume",axis=1)
Y=df["volume"]

#### Convert Training and Test Data to numpy float32 arrays


In [18]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=3)

X_train = X_train.astype('float32')
Y_train = Y_train.astype('float32')
X_test = X_test.astype('float32')
Y_test = Y_test.astype('float32')

In [19]:
X_train.shape

(800, 4)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [20]:
from sklearn.preprocessing import Normalizer

In [21]:
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)

In [22]:
X_test = transformer.fit_transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [23]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

In [24]:
xw_matmul = tf.matmul(X_train, w)

2.Define a function to calculate prediction

In [25]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [26]:
def loss(y_actual, y_predicted):
    
    avg = tf.reduce_mean(tf.squared_difference(y_actual,y_predicted))
    
    return avg

def cost(y_actual, y_predicted):
    
    cost = tf.reduce_sum(tf.pow(y_predicted-y_actual, 2)) / (2 * len(y_actual))
    return cost

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [27]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
        #current_cost = cost(y_actual, current_prediction)
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [28]:
for i in range(100):
    
    w, b = train(X_train, Y_train, w, b)
    print('Current Loss on iteration', i, loss(Y_train, prediction(X_train, w, b)).numpy())

Current Loss on iteration 0 197.8092
Current Loss on iteration 1 195.7881
Current Loss on iteration 2 193.92535
Current Loss on iteration 3 192.2088
Current Loss on iteration 4 190.62651
Current Loss on iteration 5 189.1686
Current Loss on iteration 6 187.82487
Current Loss on iteration 7 186.58646
Current Loss on iteration 8 185.44518
Current Loss on iteration 9 184.39331
Current Loss on iteration 10 183.42395
Current Loss on iteration 11 182.53052
Current Loss on iteration 12 181.70723
Current Loss on iteration 13 180.94847
Current Loss on iteration 14 180.24915
Current Loss on iteration 15 179.60464
Current Loss on iteration 16 179.01076
Current Loss on iteration 17 178.46338
Current Loss on iteration 18 177.95891
Current Loss on iteration 19 177.49393
Current Loss on iteration 20 177.06548
Current Loss on iteration 21 176.67062
Current Loss on iteration 22 176.30667
Current Loss on iteration 23 175.97137
Current Loss on iteration 24 175.6622
Current Loss on iteration 25 175.37727
C

### Get the shapes and values of W and b

In [29]:
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[1.2978185]
 [1.3030543]
 [1.2847368]
 [1.3140031]]
Bias:
 [2.599959]


In [30]:
xt1 = [X_test[1]]

### Model Prediction on 1st Examples in Test Dataset

In [31]:
y_predict = prediction(xt1,w,b)

In [32]:
y_predict

<tf.Tensor: id=5113, shape=(1, 1), dtype=float32, numpy=array([[5.199538]], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [33]:
iris = pd.read_csv('Iris.csv')

In [34]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [35]:
iris.Species.unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [36]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
iris['Species'] = le.fit_transform(iris.Species)

In [37]:
iris.head()


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0


In [38]:
dummy = pd.get_dummies(iris.Species)
iris1 = pd.concat([iris,dummy], axis=1)

In [39]:
iris1.drop('Species',axis=1,inplace=True)


In [40]:
iris1.drop('Id', axis=1,inplace=True)

In [41]:
iris1.head(5)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,0,1,2
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [42]:
X = iris1.iloc[: ,0:4]
Y= iris1.iloc[:,4:7]
X.shape


(150, 4)

In [43]:
Y.shape

(150, 3)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [44]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape((784,),input_shape=(28,28,)))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3,input_shape=(4,), activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Model Training 

In [45]:

import numpy as np

In [46]:
X_train, X_test,Y_train, Y_test = train_test_split(X , Y , test_size=0.3,random_state=3)

X_train = X_train.astype('float32')
Y_train = Y_train.astype('float32')
X_test = X_test.astype('float32')
Y_test = Y_test.astype('float32')


In [47]:
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [48]:
Y_test

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [49]:
Y_train

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1

In [50]:
model.fit(X_train, Y_train, 
          validation_data=(X_test, Y_test), epochs=100)

Instructions for updating:
Colocations handled automatically by placer.
Train on 105 samples, validate on 45 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
105/105 [==============================] - 1s 5ms/sample - loss: 1.5021 - acc: 0.3810 - val_loss: 2.8970 - val_acc: 0.3333
Epoch 2/100
105/105 [==============================] - 0s 129us/sample - loss: 1.4358 - acc: 0.3619 - val_loss: 2.8166 - val_acc: 0.3333
Epoch 3/100
105/105 [==============================] - 0s 187us/sample - loss: 1.3995 - acc: 0.3619 - val_loss: 2.7374 - val_acc: 0.3333
Epoch 4/100
105/105 [==============================] - 0s 122us/sample - loss: 1.3257 - acc: 0.3810 - val_loss: 2.6657 - val_acc: 0.3333
Epoch 5/100
105/105 [==============================] - 0s 103us/sample - loss: 1.3008 - acc: 0.3905 - val_loss: 2.5880 - val_acc: 0.3333
Epoch 6/100
105/105 [==============================] - 0s 161us/sample - loss: 1.2697 - acc: 0.3619 - val_loss: 2.5157 - val_acc: 0.3333
Epoch 7/100
105

105/105 [==============================] - 0s 40us/sample - loss: 0.6076 - acc: 0.7143 - val_loss: 0.5588 - val_acc: 0.7333
Epoch 57/100
105/105 [==============================] - 0s 137us/sample - loss: 0.6022 - acc: 0.7238 - val_loss: 0.5519 - val_acc: 0.7333
Epoch 58/100
105/105 [==============================] - 0s 157us/sample - loss: 0.6162 - acc: 0.7333 - val_loss: 0.5445 - val_acc: 0.7556
Epoch 59/100
105/105 [==============================] - 0s 61us/sample - loss: 0.5985 - acc: 0.7524 - val_loss: 0.5377 - val_acc: 0.7556
Epoch 60/100
105/105 [==============================] - 0s 85us/sample - loss: 0.5959 - acc: 0.7143 - val_loss: 0.5315 - val_acc: 0.7556
Epoch 61/100
105/105 [==============================] - 0s 139us/sample - loss: 0.5862 - acc: 0.7238 - val_loss: 0.5247 - val_acc: 0.7556
Epoch 62/100
105/105 [==============================] - 0s 82us/sample - loss: 0.5752 - acc: 0.7048 - val_loss: 0.5198 - val_acc: 0.7778
Epoch 63/100
105/105 [=============================

### Model Prediction

In [51]:
xt1 = X_test[0:1]

In [52]:
model.predict(xt1)

array([[0.9255732 , 0.07181508, 0.0026117 ]], dtype=float32)

In [53]:
Y_test[0:1]

array([[1., 0., 0.]], dtype=float32)

### Save the Model

In [54]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_v1 (Batc multiple                  16        
_________________________________________________________________
dense (Dense)                multiple                  15        
Total params: 31
Trainable params: 23
Non-trainable params: 8
_________________________________________________________________


In [55]:
model.save_weights('iris_1.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?